# 01 - Objects and Classes

We will try to refer to instances of classes as **instances** and not **objects** throughout this course because pretty much everything is an object.

The class itself is also called a `type`. For example, we call `list` a type but it's also a `list` class.

The type of our classes (not instances) are of type: `type`. This includes things like the `list` class - we'll look into all of this later.

The type of `type` is also of type: `type`. Again, we'll look into all of this later.

# 02 - Class Attributes

Everything in this subsection is related to the class itself, **not the instances of the class**.

We can **get** the attributes of **any object** in general, not just classes, using the `getattr` function:
```python
getattr(object_symbol, attribute_name, optional_default)
```
- The shorthand dot notation is identical to this except you can't specify defaults with dot notation.
- When bounded to a class, they are often called **data attributes** or **class attributes** in contrast to **instance attributes**.

Here's a simple example:

In [14]:
class MyClass:
    language = 'Python'
    version = '3.6'

print(getattr(MyClass, 'language'))
print(getattr(MyClass, 'x', 'N/A'))

Python
N/A


We can **set** the attributes of **any object** in general, not just classes, using the `setattr` function:
```python
getattr(object_symbol, attribute_name, attribute value)
```
- Since python is a dynamic language, we can define attributes in our code to modify our classes at runtime.

Here's a simple example:

In [15]:
class MyClass:
    language = 'Python'
    version = '3.6'

setattr(MyClass, 'version', '3.7')
print(getattr(MyClass, 'version'))

setattr(MyClass, 'x', 100)
print(getattr(MyClass, 'x'))

3.7
100


We can **delete** the attributes of **any object** (usually), not just classes, using the `delattr` function:
```python
delattr(object_symbol, attribute_name)
```
- We can also use the `del` keyword which we've seen in the past for removing keys from dictionaries. This fact will become more relevant after reading down below.

**Where is the state stored?**

- The state of a class is stored in a dictionary accessed with `__dict__`.
- This returns a **mappingproxy** which is a hashmap (dictionary) but not a python `dict`. It is essentially a read-only dictionary that's only mutable via `setattr`. This is in contrast to the state of an instance accessed by `__dict__` which *is* a python `dict` and is therefore read-and-write.
- This dictionary is often called the **class namespace**.

In [16]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              'language': 'Python',
              'version': '3.7',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None,
              'x': 100})

While we can get (but not set) attributes via this dictionary, it's better to use `getattr` or the dot notation as classes can sometimes have attributes that **don't** show up in this dictionary e.g. `__name__`.

In [17]:
MyClass.__dict__['language']

'Python'

# 03 - Callable Class Attributes

Class attributes can be any object type, including callables such as functions. These are **not** instance methods as they do not take `self` in their argument. They are typically called **function attributes**.

In [19]:
class Program:
    language = 'Python'
    
    def say_hello():
        print(f'Hello from {Program.language}!')

Since this function is an **attribute**, we can use `getattr` or dot notation:

In [21]:
Program.say_hello()
getattr(Program, 'say_hello')()

Hello from Python!
Hello from Python!


# 04 - Classes are Callable

When we use the `class` keyword, Python automatically adds behaviours to the class.

It makes it a callable with a return value that is an object which we call an **instance**.

The **instance** has its own **_distinct_** namespace which is separate to the namespace of the class itself.

`__dict__` on the instances returns the instance's **local** namespace while `__dict__` on the class returns the class' namespace.

In [28]:
class Program:
    language = 'Python'
    
    def say_hello():
        print(f'Hello from {Program.language}!')

In [29]:
Program.__dict__

mappingproxy({'__module__': '__main__',
              'language': 'Python',
              'say_hello': <function __main__.Program.say_hello()>,
              '__dict__': <attribute '__dict__' of 'Program' objects>,
              '__weakref__': <attribute '__weakref__' of 'Program' objects>,
              '__doc__': None})

In [30]:
p.__dict__

{}

We should **always** use the `type` function over the `__class__` attribute to access the original class.

This is because `__class__` is just an attribute that can be manually modified:

In [34]:
class MyClass:
    __class__ = str

m = MyClass()

In [35]:
print(m.__class__)
print(type(m))

<class 'str'>
<class '__main__.MyClass'>


The `isinstance` function *is* affected by our modification too. This is more related to **metaprogramming** so we'll cover it in more detail later, but to demonstrate:

In [39]:
print(isinstance(m, str))
print(isinstance(m, MyClass))

True
True


But we should definitely use **isinstance** when it comes to inheritance which we'll also look at later.

# 05 - Data Attributes

Let's illustrate the differences between the class and instance namespaces with an example. 

We will demonstrate how performing a lookup for an instance attribute will look in the local namespace, and if it doesn't find it there, it will look in the class namespace:

In [47]:
class BankAccount:
    apr = 1.2

In [52]:
acc_1 = BankAccount()
acc_2 = BankAccount()

acc_1.__dict__

{}

In [50]:
acc_1.apr

1.2

If we set the **instance attribute**, only that instance's namespace will be affected. Other instance namespaces and the class namespace itself will be unaffected.

In [56]:
acc_1.apr = 0

acc_1.__dict__, acc_2.__dict__

({'apr': 0}, {})

If we get the **instance attribute** back, we'll always search in the local namespace first before moving up the inheritance tree:

In [57]:
acc_1.apr, acc_2.apr

(0, 1.2)

# 06 - Function Attributes

So far, we have been dealing with non-callable attributes. When attributes are actually functions, things behave differently.

Let's look at the difference between viewing this function through the class vs. through an instance:

In [5]:
class Person:
    def say_hello():
        print('Hello!')

In [6]:
Person.say_hello

<function __main__.Person.say_hello()>

In [17]:
p = Person()
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x00000229C2158A60>>

In [27]:
type(Person.say_hello), type(p.say_hello)

(function, method)

**Bound methods** 

- These are different from functions.
- They can only be called bound methods once an instance has been made which binds it to the method - otherwise, they're just functions.
- The object that they are bound to is passed (injected) to the method as its **first parameter**.
- Methods are objects and like any objects in Python, it has attributes:
    - `__self__`: the instance the method is bound to.
    - `__func__`: the original function defined in the class. This is how the method knows what function to call.

We cannot call `say_hello` as a bound method since the function hasn't been set up to deal with taking the instance as its first argument. We'll fix that:

In [23]:
class Person:
    language = 'Python'
    def say_hello(obj, name):
        print(f'Hello! {name}! I am {obj.language}.')

As can be seen below, calling the bound method is identical to calling the function with the instance. The `obj.language` used was not found in the local namespace so it was searched for in the class namespace.

By convention, we should use `self` for the name of the instance which we shall do from now on.

In [25]:
python = Person()
python.say_hello('John')
Person.say_hello(python, 'John')

Hello! John! I am Python.
Hello! John! I am Python.


We can add methods *after* creating the class definition (monkeypatching). We must remember to add `self` as the first argument to ensure it's treated like a method:

In [51]:
Person.say_goodbye = lambda self, name: print(f'Goodbye {name}! From {self.language}.')
python.say_goodbye

<bound method <lambda> of <__main__.Person object at 0x00000229C2007A00>>

In [52]:
python.say_goodbye('John')

Goodbye John! From Python.


This runtime initialisation **only affects the bound instance**. If a new instance is created, it will not have access to this method, because `say_goodbye` is only bound to `python`.

In [57]:
java = Person()
java.say_goodbye()

AttributeError: 'Person' object has no attribute 'say_goodbye'

# 07 - Initializing Class Instances

We've seen that when we instantiate a class, we
1. create a **new instance** of the class
2. **initialise** the namespace of the instance: `obj.__dict__ -> {}`

We can "intercept" both the creating and initialization phases, by using special methods `__new__` and `__init__`.

We'll come back to `__new__` later. For now we'll focus on `__init__`.

But as you already might know, we can perform our own initialisation. We do this with `__init__` function which is a **class attribute** in the **class namespace**. It is only when we've created the instance and created the instance namespace (`obj.__dict__ -> {}`) that the `__init__` function is called as a bound method.

In [44]:
class MyClass:
    def __init__(self, version):
        self.version = version

In [46]:
m = MyClass(3.7)
m.version

3.7

In [49]:
MyClass.__init__(m, 3.8)
m.version

3.8

# 08 - Creating Attributes at Run-Time

Another way we can create and bind a method to an instance at runtime is using `MethodType` from the `types` library:
```python
MethodType(function, object)
```
where the `function` is the function we want to bind and the `object` is that object we want to bind it to.

In [59]:
from types import MethodType

class Person:
    language = 'Python'

p = Person()
p.say_hello = MethodType(lambda self, name: f'Hello {name}! I am {self.language}.', p)

In [60]:
p.say_hello('John')

'Hello John! I am Python.'

As seen before, this runtime initialisation **only affects the bound instance**. If a new instance is created, it will not have access to this method, because `say_hello` is only bound to `python`.

In [63]:
java = Person()
java.say_hello('John')

AttributeError: 'Person' object has no attribute 'say_hello'

With this approach, we can define different functions and register them to different instances. The only common thing we would need among the different instances is a register manager written in the original class which would handle the assignment/binding and maybe some error handling. 

See original notebook for an example.

# 09 - Properties

So far, what we've seen in our classes are "bare" attributes which we access directly via dot notation (or `getattr`, `setattr`). These are always publicly accessible in python but we can prepend attributes with an underscore to indicate that it's private. 

Often a bare attribute works just fine, and if, later, we decide we need to manage getting/setting/deleting the attribute value, we can switch over to properties without breaking our class interface, because properties use the same dotted notation. This is unlike languages like Java - and hence why in those languages it is recommended to **always** use getter and setter functions. *Not so* in Python!

We've often seen properties being used as decorators but they are just classes which we can instantiate. Here's an example:

In [4]:
class Person:
    def __init__(self, name):
        self.name = name  # note how we are actually setting the value for name using the property!
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
            
    name = property(fget=get_name, fset=set_name)  # this is a class attribute

In [5]:
p = Person('John')
p.name = 'Eric'
print(p.name)

Eric


`p.name` will first look in the local (instance) namespace and it won't find it there.. So it looks in the class namespace and finds it.

Below, we further prove that `name` is a class attribute and not an instance attribute. 

In [9]:
print(p.__dict__)

{'_name': 'Eric'}


The property object will call the function in `fget` or `fset` depending on how the class attribute `name` was used.

How does python actually implement `property` to do these bound getter and setter methods? We'll see later on..

The `property` class can be used in another way. It has getter, setter and deleter methods which take the relevant callable as an argument: 

In [24]:
class Person:
    def __init__(self, name):
        self.name = name  
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
            
    name = property()
    name = name.getter(get_name)
    name = name.setter(set_name)

In [23]:
p = Person('John')
p.name = 'Eric'
print(p.name)

Eric


Or in this way:

In [26]:
class Person:
    def __init__(self, name):
        self.name = name  
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
            
    name = property(get_name)  # name is passed positionally to property

This form makes it easier to see the decorator pattern emerging of `x = a(x)`. We would just need to replace the names of our getters and setters with the name of our property. We haven't included the setter line yet which will become clear in a moment.

# 10 - Property Decorators

Applying the @property decorator to the getter is simple to understand: 
```python
    def name(self):
        return self._name

    name = property(name)  
```
is equivalent to
```python
    @property
    def name(self):
        return self._name
```

But for the setter, we will need to follow the order of execution carefully. At the end of the decorated function, `name` is a **property instance** with the getter function defined/applied to it. If we want to apply the setter function, we must use the `.setter` method on our property instance. Therefore, we get:
```python
    def set_name(self):
        return self._name

    name = name.setter(set_name)
```
which is equivalent to:
```python
    @name.setter
    def set_name(self):
        return self._name
```

We can call the decorated function `name` instead of `set_name` because the `name` in the decorator `@name.setter` is pointing to the previous property instance. But we can't write: 
```python
    def name(self):
        return self._name

    name = name.setter(name)
```
because `name` in `name.setter` gets redefined to the function above. Therefore, since that function has no method called `setter`, we would get an error with this approach. This explanation was included here to illustrate that the non-decorator vs decorator approaches are not entirely equivalent.

One final note: if we want more information on a property, we must write the docstring **in the getter** for it to be visible with the `help()` function.

The final answer:

In [53]:
class Person:
    def __init__(self, name):
        self.name = name  

    @property
    def name(self):
        """This is the name"""
        return self._name

    @name.setter
    def name(self, value):
        self._name = value

In [54]:
p = Person('John')
p.name = 'Eric'
print(p.name)

Eric


In [55]:
help(Person.name)

Help on property:

    This is the name



We now have a property called `name` with both getter and setter applied.

How do we create a class with a setter but *not* a getter? And how would we provide a docstring in that case? As we saw above, the setter decorator relied on the property instance created from decorating the getter.

One solution is to instantiate the property class with no arguments:

In [56]:
class Person:
    name = property(doc='This is a write-only property') 
    
    @name.setter
    def name(self, value):
        print('setter called')
        self._name = value

In [57]:
p = Person()
p.name = 'Eric'

setter called


In [58]:
help(Person.name)

Help on property:

    This is a write-only property



# 11 - Read-Only and Computed Properties

To make a read-only property, all we need to do is implement a getter but not a setter. Of course the property can still be modified via the "private" attribute but there's nothing in Python to prevent that.

Computed properties are when the property getter performs some computation but can still be deemed a property as opposed to a method.

For example, given a radius of a circle, the area can be a computed property. 

We may also want to **cache** computed properties. Here we store and access the value as a property but any time we change anything that affects the computation, we **invalidate the cache** and re-calculate the property. For example, if the radius changes, the area must be re-calculated. Here's a simple implementation:

In [70]:
from math import pi

class Circle:
    def __init__(self, radius):
        self._radius = radius
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        self._area = None  # invalidate cache
        self._radius = value
        
    @property
    def area(self):
        if self._area is None:  # check to see if cache invalidated
            print('calculating area..')
            self._area = pi * (self.radius ** 2)
        return self._area

In [71]:
c = Circle(1)
c.area
c.area
c.area

calculating area..


3.141592653589793

In [72]:
c.radius = 2
c.area

calculating area..


12.566370614359172

In the original notebook, you can find a more complex example of caching computed properties in a class that downloads webpages. But the approach is principally identical to the above.

# 12 - Deleting Properties

Just like we can delete an attribute from an instance object, we can also delete a property from an instance object.

Note that this action simply runs the deleter method, but the property remains defined **on the class**. It does not remove the property from the class, instead it is generally used to remove the property value from the **instance**.

Properties, like attributes, can be deleted by using the `del` keyword, or the `delattr` function. But the property instance **still exists**. Therefore, we can run the setter to recreate it after deleting:

In [75]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        print('getting name property value...')
        return self._name
    
    @name.setter
    def name(self, value):
        """Person name"""
        print(f'setting name property to {value}...')
        self._name = value
    
    @name.deleter
    def name(self):
        # delete the underlying data
        print('deleting name property value...')
        del self._name

In [76]:
p = Person('Alex')

setting name property to Alex...


In [77]:
p.name

getting name property value...


'Alex'

In [78]:
del p.name

deleting name property value...


In [79]:
p.name = 'John'

setting name property to John...


Before we wrap up the properties section, we may have the following question:

Q: We saw how to use the `property` class to define **instance** properties. Can we create **class** properties? 

A: Yes, and they're called **metaclasses** which we'll cover later on..

# 13 - Class and Static Methods

Earlier, we saw how we can define a plain function in a class:

In [96]:
class MyClass:
    def hello():
        print('Hello')

This is **not bound** to our class.

In [97]:
MyClass.hello

<function __main__.MyClass.hello()>

##### Instance Method

We also saw how we can define a regular function that will be **bound to an instance** -> instance method:

In [110]:
class MyClass:
    def hello(self):  # instance is injected
        print(f'Hello from {self}')

m = MyClass()
m.hello  # hello bounded to instance `m`

<bound method MyClass.hello of <__main__.MyClass object at 0x000001B7173410F0>>

In [111]:
m.hello()

Hello from <__main__.MyClass object at 0x000001B7173410F0>


##### Class Method

But can we define a function in a class that will **always be bound to the class** and *never* the instance?

Yes, using `@classmethod`. `hello()` is no longer a regular function inside `MyClass`:

In [112]:
class MyClass:
    @classmethod
    def hello(cls):  # class is injected
        print(f'Hello from {cls}')

m = MyClass()
m.hello  # hello bounded to instance `m`

<bound method MyClass.hello of <class '__main__.MyClass'>>

In [113]:
m.hello()

Hello from <class '__main__.MyClass'>


Class methods have **no knowledge** of their instances.

##### Static Method

Can we define a function in a class that will **never be bound to any object when called**?

Yes, using `@staticmethod`. Although we say "method", it is actually just a regular function *because* it's unbounded.

In [114]:
class MyClass:
    @staticmethod
    def hello():
        print(f'Hello')

m = MyClass()

MyClass.hello()
m.hello()

Hello
Hello


In [115]:
MyClass.hello, m.hello

(<function __main__.MyClass.hello()>, <function __main__.MyClass.hello()>)

Static methods are often discouraged as they can be placed at the module level if they're not bound to anything. But, sometimes it may make more sense to collect them in a class.

#### Example

Let's see a more concrete example of using these different method types.

We're going to create a `Timer` class that will allow us to get the current time (in both UTC and some timezone), as well as record start/stop times.

We want to have the same timezone for all instances of our `Timer` class with an easy way to change the timezone for all instances when needed.

If you need to work with timezones, I recommend you use the `pyrz` 3rd party library. Here, I'll just use the standard library, which is definitely not as easy to use as `pytz`.

I have copied and pasted the final implementation from the original notebook for the sake of brevity; please see that if you prefer to see the gradual build up to the final implementation.

In [118]:
from datetime import datetime, timezone, timedelta


class TimerError(Exception):
    """A custom exception used for Timer class"""
    # (since """...""" is a statement, we don't need to pass)
    
class Timer:
    tz = timezone.utc  # class variable to store the timezone - default to UTC
    
    def __init__(self):
        # use these instance variables to keep track of start/end times
        self._time_start = None
        self._time_end = None
        
    @staticmethod
    def current_dt_utc():
        """Returns non-naive current UTC"""
        return datetime.now(timezone.utc)
    
    @classmethod
    def set_tz(cls, offset, name):
        cls.tz = timezone(timedelta(hours=offset), name)
        
    @classmethod
    def current_dt(cls):
        return datetime.now(cls.tz)
    
    def start(self):
        # internally we always non-naive UTC
        self._time_start = self.current_dt_utc()
        self._time_end = None
        
    def stop(self):
        if self._time_start is None:
            # cannot stop if timer was not started!
            raise TimerError('Timer must be started before it can be stopped.')
        self._time_end = self.current_dt_utc()
        
    @property
    def start_time(self):
        if self._time_start is None:
            raise TimerError('Timer has not been started.')
        # since tz is a class variable, we can just as easily access it from self
        return self._time_start.astimezone(self.tz)  
        
    @property
    def end_time(self):
        if self._time_end is None:
            raise TimerError('Timer has not been stopped.')
        return self._time_end.astimezone(self.tz)
    
    @property
    def elapsed(self):
        if self._time_start is None:
            raise TimerError('Timer must be started before an elapsed time is available')
            
        if self._time_end is None:
            # timer has not ben stopped, calculate elapsed between start and now
            elapsed_time = self.current_dt_utc() - self._time_start
        else:
            # timer has been stopped, calculate elapsed between start and end
            elapsed_time = self._time_end - self._time_start
            
        return elapsed_time.total_seconds()

In [119]:
from time import sleep

t1 = Timer()
t1.start()
sleep(2)
t1.stop()
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2024-06-27 16:38:39.213091+00:00
End time: 2024-06-27 16:38:41.222213+00:00
Elapsed: 2.009122 seconds


In [120]:
t2 = Timer()
t2.start()
sleep(3)
t2.stop()
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2024-06-27 16:38:41.246210+00:00
End time: 2024-06-27 16:38:44.254180+00:00
Elapsed: 3.00797 seconds


So our timer works. Furthermore, we want to use `MST` throughout our application, so we'll set it, and since it's a class level attribute, we only need to change it once. **This is a key takeaway from this example.**

In [121]:
Timer.set_tz(-7, 'MST')

In [122]:
print(f'Start time: {t1.start_time}')
print(f'End time: {t1.end_time}')
print(f'Elapsed: {t1.elapsed} seconds')

Start time: 2024-06-27 09:38:39.213091-07:00
End time: 2024-06-27 09:38:41.222213-07:00
Elapsed: 2.009122 seconds


In [123]:
print(f'Start time: {t2.start_time}')
print(f'End time: {t2.end_time}')
print(f'Elapsed: {t2.elapsed} seconds')

Start time: 2024-06-27 09:38:41.246210-07:00
End time: 2024-06-27 09:38:44.254180-07:00
Elapsed: 3.00797 seconds


#### Python Builtin and Standard Types

This small section is just to illustrate that not all types that we use are directly available in the builtins.

Builtin: `int`, `str`, `list`, `tuple`, ...
Not Builtin: `functions`, `modules`, `generators`

Calling `type()` on any of the instances of the builtins will return a class that *is* directly available in Python

In [128]:
l = [1, 2, 3]
type(l) is list

True

But calling `type()` on any non-builtins will return a class that *is not* directly available in Python. 

But we can access them using the `types` library:

In [129]:
import types

def my_func():
    pass

type(my_func) is types.FunctionType

True

# 14 - Class Body Scope

The class body is a scope and therefore has it's own namespace.

However, functions defined inside the class are not nested in the body scope - instead they are nested in whatever scope the class itself is in - usually the module.

Let's demonstrate this with an example:

In [131]:
class Python:
    kingdom = 'animalia'
    phylum = 'chordata'
    family = 'pythonidae'

    def __init__(self, species):
        self.species = species

    def say_hello(self):
        return 'ssss...'

<img src=s2-images/2.1.png width=700 />

**Scopes**

Recall that symbols point to **objects**


- module1.py:
    - `Python` (symbol)
    - `p` (symbol)
    - `__init__` (function object)
    - `say_hello` (function object)
 
- `Python`:
    - `kingdom` (symbol)
    - `phylum` (symbol)
    - `family` (symbol)
    - `__init__` (symbol)
    - `say_hello` (symbol)

What's so significant about the `__init__` and `say_hello` function objects existing in the enclosing scope of the class and not the class itself?

When Python looks for a symbol in a function, it will **not** use the class body scope and instead use the module scope.

Here's an example:

Writing something like this will not work:

In [1]:
class Language:
    MAJOR = 3
    MINOR = 7
    REVISION = 4
    
    @classmethod
    def cls_version(cls):
        return '{}.{}.{}'.format(MAJOR, MINOR, REVISION)

In [2]:
Language.cls_version()

NameError: name 'MAJOR' is not defined

The scope that is used is always the enclosing scope, which is not necessarily the module scope. If it doesn't find it there, it will look expand out one scope further, and so on.

In [7]:
MAJOR = 0
MINOR = 0
REVISION = 1

def gen_class():
    MAJOR = 0
    MINOR = 4
    REVISION = 2
    
    class Language:
        MAJOR = 3
        MINOR = 7
        REVISION = 4

        @classmethod
        def version(cls):
            return '{}.{}.{}'.format(MAJOR, MINOR, REVISION)
        
    return Language

In [8]:
my_cls = gen_class()

In [9]:
my_cls.version()

'0.4.2'

The fact that `MAJOR`, `MINOR` and `REVISION` are interacting with variables from an outer scope makes `version(cls)` a **closure**:

In [10]:
import inspect

inspect.getclosurevars(my_cls.version)

ClosureVars(nonlocals={'MAJOR': 0, 'MINOR': 4, 'REVISION': 2}, globals={}, builtins={'format': <built-in function format>}, unbound=set())

Let's go through one final example:

In [12]:
name = 'Guido'

class MyClass:
    name = 'Raymond'
    list_1 = [name] * 3
    list_2 = [name.upper() for i in range(3)]
    
    @classmethod
    def hello(cls):
        return '{} says hello'.format(name)

We know that the `name` in `hello(cls)` will find `Guido` because `hello(cls)` is a **function** and functions live in the outer scope:

In [13]:
MyClass.hello()

'Guido says hello'

We know that `list_1` will find `Raymond` as `list_1` is **not a function** so it will search in the current scope:

In [14]:
MyClass.list_1

['Raymond', 'Raymond', 'Raymond']

But, what does `list_2` return?

In [15]:
MyClass.list_2

['GUIDO', 'GUIDO', 'GUIDO']

Remember what we discussed about comprehensions in an earlier part of this course?

**List comprehensions are thinly veiled functions!** The same goes for generator comprehensions, dictionary comprehensions, set comprehensions etc.

So the `name` in `name.upper()` is a free variable pointing to the outer (module) scope.